In [0]:
#pip install langchain langchain_community mlflow -qU

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
 #dbutils.library.restartPython() 

In [0]:
import pandas as pd

from sqlalchemy import create_engine

from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_core.tools import Tool, BaseTool

#from langchain_openai import ChatOpenAI

In [0]:

df = pd.read_csv("hf://datasets/aibabyshark/insurance_customer_support_QA_result/insurance_customer_support_conversation_with_qa_result.csv")

In [0]:
engine = create_engine("sqlite:///qa.db")
df.to_sql("qa_table",engine, if_exists='replace', index=False)
db = SQLDatabase.from_uri("sqlite:///qa.db")

In [0]:
from langchain.chat_models import ChatDatabricks

chat_model = ChatDatabricks(endpoint="databricks-meta-llama-3-1-405b-instruct", max_tokens=4000)


In [0]:
sql_agent = create_sql_agent(chat_model, db=db, agent_type="zero-shot-react-description", verbose=False)

sql_tool = Tool.from_function(
    name="sql_agent", 
    func=sql_agent.run, 
    description="useful for answering questions by using SQL",
    handle_tool_error = True,
    verbose = False, 
    error_message = "Error running sql code",
) 
